<a href="https://colab.research.google.com/github/vilsonrodrigues/Projeto_Orientacao_Academica/blob/master/Projeto_Orientacao_Academica_Tratamento_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Por @vilsonrodrigues



# <center><font color='blue'>Projeto de Orientação Acadêmica</font>




<img src='https://ufrn.br/resources/documentos/identidadevisual/logotipo/logotipo_flat.png'>

## Parte I - Apresentação do Problema
### [1. Apresentação do Problema](#apresentacao)
## Parte II - Preparativos Iniciais
### [1. Bibliotecas](#bibliotecas)
### [2. Classe](#classe)
### [3. Funções](#funcoes)
### [4. Leituras da Bases](#leiturabase)
## Parte III - Desenvolvimento
### [1. Entendimento de Negócio](#negocio)
### [2. Entendimento de Dados](#enten_dados)
### [3. Preparação de Dados](#preparo)
### [4. Métricas de Avaliação](#metricas)
### [5. Modelagem](#modelagem)
### [6. Avaliação de Modelo](#avaliacao)
### [7. Conclusão e Recomendações](#conclusao)
### [8. Implantação de Modelos](#implantacao)


# Parte I - Apresentação do Problema

### **Meta**
Nossa meta é prever através de Machine Learning as probabilidades de um estudante ter sucesso em uma disciplina dado o histórico escolar na Universidade.

Nossos dados foram transformados para ficarem de uma forma génerica, assim, não dependemos de saber qual foi a matéria cursada anteriormente.

Com essa plataforma, os professores que orientam alunos, terão mais segurança e uma comprovação com base no Machine Learning que estudantes que tiveram um histórico semelhante e seguiram esse caminho tiveram mais êxito do que seguir por este outro.

Os dados foram coletados do período de 2015.1 até 2019.1.

**Esse notebook foca totalmente em analisar os dados e montar uma estratégia  de gerar dados genéricos, de modo que fiquem independente de qual disciplina o aluno curso no semeste anterior.**

O trabalho continua neste [notebook](https://), onde são gerados modelos de Machine Learning com dados genéricos.

### **Fonte dos Dados**
Dados foram extraídos do portal SIGAA e fornecidos pela Escola de Ciências e Tecnlogia (ECT).

### **Referência**
Hands On: Machine Learning with Scikit-Learning e Tensor Flow



# Parte II - Preparativos Iniciais

## 1. Bibliotecas
<a id='bibliotecas'></a>
#### [1.1 Instalações](#instalacao)
#### [1.2 Importações](#importacao)


### 1.1. Instalação
<a id='instalacao'></a>

In [0]:
!pip install pandas -U
!pip install scikit-learn -U
!pip install numpy -U
!pip install joblib==0.14.1

### 1.2. Importação
<a id='importacao'></a>

In [0]:
import pandas as pd
import numpy as np
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [0]:
import pandas as pd
import numpy as np

#Persistência em disco
import joblib
import pickle

#Visualização
import seaborn as sns
import matplotlib.pyplot as plt

#Transformadores e Modelador
from sklearn.pipeline          import Pipeline
from sklearn.base              import BaseEstimator, TransformerMixin

#Selecao de modelo
from sklearn.model_selection   import train_test_split
from sklearn.model_selection   import KFold
from sklearn.model_selection   import cross_val_score
from sklearn.model_selection   import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import chi2

#modo não-perturbe
import warnings
warnings.filterwarnings("ignore")

##2. Classe
<a id='classe'></a>


Classe para realizar a geração de dados genéricos

In [0]:
class geraDadosGenericos(TransformerMixin):  
  def __init__(self, carga):
    self.carga = carga

	#nao irei fazer nenhum treinamento	
  def fit(self, X, y = None):
    return self    

  #-------------------------funcoes de geracao de dados da classe---------------------------------

  def filtraSituacoes(self, df_notas):
    #conversao para string
    df_notas['matricula'] = df_notas['matricula'].astype(str)
    df_notas['periodo']   = df_notas['periodo'].astype(str)

    #substitui todas as situacoes de reprovado e aprovado para ser apenas um tipo
    df_notas.loc[df_notas['situacao'].str.contains('REPROVADO'), 'situacao'] = 'REPROVADO'
    df_notas.loc[df_notas['situacao'].str.contains('APROVADO'), 'situacao']  = 'APROVADO'

    #seleciona as situacoes de interesse que são aprovados e reprovados
    df_notas = df_notas.loc[(df_notas['situacao'].str.contains('APROVADO')) |
                            (df_notas['situacao'].str.contains('REPROVADO')) 
                          ] 	
    #tem disciplina que só contam para a horas complementares, não tem nota
    df_notas = df_notas.loc[ ~(df_notas['nota'].str.contains('-')), :]

    #conversao para de string para float
    df_notas['nota'] = df_notas['nota'].astype(float)
    return df_notas


  def converteString(self, df_notas):
    df_notas['periodo']   = df_notas['periodo'].astype(str)
    return df_notas


  def concatenaSemestre(self, df_notas):
    #pega o ano mais alto que tem na matricula, indicando o semestre do df
    ano_df = df_notas['matricula'].str[0:4].max()
    df_notas['semestre'] = str(ano_df) + '.' + str(df_notas['periodo'].iloc[0])
    #apagando a coluna periodo, nao serve mais
    df_notas.drop(columns = ['periodo'], inplace=True)
    #conversao para float
    df_notas['semestre'] = df_notas['semestre'].astype(float)
    df_notas['matricula'] = df_notas['matricula'].astype(int)
    return df_notas

  #adiciona quanto tempo o estudante esta com a matricula vingente 
  def anosMatriculado(self, df_notas): 
    df_notas['matricula'] = df_notas['matricula'].astype(str)
    df_notas['anoIngresso'] = df_notas['matricula'].str[0:4]
    df_notas['anoIngresso'] = df_notas['anoIngresso'].astype(float)
    df_notas['semestre'] = df_notas['semestre'].astype(float)
    df_notas['anosMatriculado'] = df_notas['semestre'] - df_notas['anoIngresso']
    df_notas['anosMatriculado'] = df_notas['anosMatriculado'].astype(int)
    return df_notas    


  #essa função vai criar as novas features com o desempenho do aluno no semestre 
  def geraFeaturesComDadosGenericos(self, df_notas):
    df_notas = df_notas.merge(self.carga, on = 'codigo')
    df = pd.DataFrame()
    final = pd.DataFrame()

    #media das disciplinas
    df = df_notas.groupby('matricula')['nota'].mean().reset_index()
    df.rename(columns={'nota':'mediaNotas'}, inplace = True)
    final = df

    #menor nota
    df = df_notas.groupby('matricula')['nota'].min().reset_index()
    df.rename(columns = {'nota':'menorNota'}, inplace = True)
    final = final.merge(df, on = 'matricula', how = 'outer')

    #maior nota
    df = df_notas.groupby('matricula')['nota'].max().reset_index()
    df.rename(columns = {'nota':'maiorNota'}, inplace = True)
    final = final.merge(df, on = 'matricula', how = 'outer')

    #quantidade de disciplinas cursadas
    df = df_notas.groupby('matricula')['codigo'].count().reset_index()
    df.rename(columns = {'codigo':'quantidadeDisciplinasCursadas'}, inplace=True)
    final = final.merge(df, on = 'matricula', how = 'outer')

    #quantidade de situacoes
    df = df_notas.groupby(['matricula','situacao'])['codigo'].count().reset_index()
    df.rename(columns = {'codigo':'quanSituacao'}, inplace = True)

    #quantidade de situacoes em que houve aprovacao
    aprovado = df.loc[df['situacao'] == 'APROVADO',:]
    aprovado.rename(columns = {'quanSituacao':'quantidadeAprovacoes'}, inplace = True)
    final = final.merge(aprovado, on = 'matricula', how = 'outer')

    #quantidade de situacoes em que houve reprovacao
    reprovado = df.loc[df['situacao'] == 'REPROVADO',:]
    reprovado.rename(columns = {'quanSituacao':'quantidadeReprovacoes'}, inplace = True)
    final = final.merge(reprovado, on = 'matricula', how = 'outer')
    final = final.drop(['situacao_x', 'situacao_y'], axis = 1)

    #media das aprovacoes
    aprov = df_notas.loc[df_notas['situacao'] == 'APROVADO',:]
    dfa = aprov.groupby(['matricula'])['nota'].mean().reset_index()
    dfa.rename(columns = {'nota':'mediaAprovadas'}, inplace = True)
    final = final.merge(dfa, on = 'matricula', how ='outer')

    #media das reprovacoes
    reprov = df_notas.loc[df_notas['situacao'] == 'REPROVADO',:]
    dfr = reprov.groupby(['matricula'])['nota'].mean().reset_index()
    dfr.rename(columns = {'nota':'mediaReprovadas'}, inplace = True)
    final = final.merge(dfr, on = 'matricula', how ='outer')

    #carga horaria total no semestre
    df = df_notas.groupby('matricula')['ch_total'].sum().reset_index()
    df.rename(columns = {'ch_total':'cargaHoraria'}, inplace = True)
    final = final.merge(df, on = 'matricula', how = 'outer')

    #substituindo valores nulos por 0
    final = final.fillna(0)

    #conversao de campos de objeto para inteiro
    final['quantidadeAprovacoes']  = final['quantidadeAprovacoes'].astype(int)
    final['quantidadeReprovacoes'] = final['quantidadeReprovacoes'].astype(int)

    #Criando campo taxa de sucesso, que será a porcentagem de aprovações dado o total de disciplinas
    final['taxaDeSucesso']  = np.divide(final['quantidadeAprovacoes'], final['quantidadeDisciplinasCursadas']) 

    #herdando o semestre
    final['semestre'] = df_notas['semestre']

    return final

  #--------------------fim das funcoes de geracao de dados da classe-----------------------------------------


  def transform(self, X, y = None):
    #itera a lista
    lista_df_notas = X.copy()
    lista_df_generico = []
    lista_notas = []
    lista_de_listas = []
    for df_notas in lista_df_notas:
      df_notas = self.converteString(df_notas)
      df_notas = self.filtraSituacoes(df_notas)
      df_notas = self.concatenaSemestre(df_notas)
      lista_notas.append(df_notas)
      df_generico = self.geraFeaturesComDadosGenericos(df_notas)
      df_generico = self.anosMatriculado(df_generico)
      lista_df_generico.append(df_generico)

    lista_de_listas.append(lista_df_generico)  
    lista_de_listas.append(lista_notas)
    return lista_de_listas

In [0]:
class concatenaSemestrePassadoComResultadosDoAtual(TransformerMixin):
  def __init__(self, data = None):
    return data

	#nao irei fazer nenhum treinamento	
  def fit(self, X, y = None):
    return self    

  #--------------------------------- funcoes de transformacao da classe -----------------------------------------
  def converteParaInt(self, df):
    df['matricula'] = df['matricula'].astype(int)
    return df


  def transformacoesSemestreAntecessor(self, semestreAntecessor):
    semestreAntecessor = self.converteParaInt(semestreAntecessor)
    semestreAntecessor = semestreAntecessor.drop(columns = ['cargaHoraria','taxaDeSucesso','semestre'])
    return semestreAntecessor


  def transformacoesSemestreAtual(self, semestreAtual): 
    semestreAtual = self.converteParaInt(semestreAtual)
    #seleciona os campos que vou precisar
    semestreAtual = semestreAtual[['matricula','cargaHoraria','semestre','quantidadeDisciplinasCursadas','anosMatriculado']]
    semestreAtual.rename(columns = {'cargaHoraria':"cargaHorariaAtual",
                                    'quantidadeDisciplinasCursadas':'quantidadeDisciplinasAtuais',
                                    'semestre':'semestreAtual'}, 
                                    inplace=True) 
    return semestreAtual


  def transformacoesResultadoAtual(self, resultadoAtual):  
    resultadoAtual = self.converteParaInt(resultadoAtual)
    #seleciona os campos que vou precisar
    resultadoAtual = resultadoAtual[['codigo','matricula','siape','situacao']]
    #cria campo vezes reprovado
    resultadoAtual["vezesReprovado"] = 0
    return resultadoAtual


  def mergeTriploTabelas(self, semestreAntecessor, semestreAtual, resultadoAtual):
    dfCompleto = pd.merge(pd.merge(semestreAntecessor, resultadoAtual, on = 'matricula'), 
                          semestreAtual.drop(columns = ['anosMatriculado']), 
                          on = 'matricula')
    return dfCompleto


  def insereVerificaReprovacoes(self, dfCompleto, df_teste):
    #df temporario para armazenar as consultas com resultados iguais
    dftemporario = pd.DataFrame()
    #itera na lista de notas para achar se o aluno ja reprovou em um semestre anterior ao atual
    for df in df_teste[1]:
      #se eh anterior ao atual semestre
      if dfCompleto['semestreAtual'].max() > df['semestre'].max():
        #vou juntar onde no passado do aluno ele cursou a disiciplina para verificar se ele ja foi reprovado, 
        #se aparecer um caso é porque ele ja pagou a disciplina antes, e consequentemente já reprovou
        merge = pd.merge(dfCompleto[['matricula','codigo','vezesReprovado']], df[['matricula','codigo']], 
                        on = ['matricula','codigo'], how = 'inner')
        #todas as iterações que cumprirem o criterio são acumuladas no df
        dftemporario = pd.concat([dftemporario,merge])
    #agrupa por disciplina, para caso tenha mais de uma reprovacao ele contar. Armazena o valor em vezesReprovado    
    dftemporario = dftemporario.groupby(['matricula','codigo'])['vezesReprovado'].count().reset_index().sort_values(by = 'matricula')
    #junta ao df do semestre atual onde tem o mesmo codigo e a matricula, e junta com outer para manter quando nao tiver reprovacao
    dftemporario = pd.merge(dfCompleto.drop(columns = ['vezesReprovado']), dftemporario, on = ['matricula','codigo'], how = 'outer')
    #substitui onde tiver NaN por 0
    dftemporario.fillna(0, inplace = True) 
    #converter de float para inteiro para ficar mais agradavel
    dftemporario['vezesReprovado'] = dftemporario['vezesReprovado'].astype(int)
    return dftemporario

  def concatena(self, lista):
    df = pd.DataFrame()
    for i in lista:
      df = pd.concat([df,i])
    return df.reset_index().drop(columns = ['index'])

  #--------------------------------------------------------fim----------------------------------------------------------------------

  def transform(self, X, y = None):
    df_nota_generico = X.copy()
    lista_df_completos = []
    for i in range (len(df_nota_generico[0]) - 1):
      semestreAntecessor = df_nota_generico[0][i]
      semestreAtual = df_nota_generico[0][i+1]
      resultadoAtual = df_nota_generico[1][i+1]
      semestreAntecessor = self.transformacoesSemestreAntecessor(semestreAntecessor)
      semestreAtual = self.transformacoesSemestreAtual(semestreAtual)
      resultadoAtual = self.transformacoesResultadoAtual(resultadoAtual)
      #juntando tabelas
      dfCompleto = self.mergeTriploTabelas(semestreAntecessor, semestreAtual, resultadoAtual)
      #envia a tabela com o resultado atual e os dados genericos do semestre anterior, e todos os resultados
      #verifica se nos semestres anteriores o aluno ja pagou essa disciplina, consequentemente, ja reprovou nela
      resultado = self.insereVerificaReprovacoes(dfCompleto, df_nota_generico)
      #adiciona o df do semestre a cada iteracao
      lista_df_completos.append(resultado)
    #concatena todos os semestres para retornar 1 so df
    return self.concatena(lista_df_completos)

##3. Funções
<a id='funcoes'></a>

Função que usa GridSearch para avaliar o melhor modelo disponível, retornar estatísticas dos resultados e persistir em disco os modelos gerados

In [0]:
def selecionaDisciplinasECT(df):
  disciplinasECT = df.loc[df['codigo'].str.contains(''), :]
  #caso para pegar apenas os 4 primeiros semestres
  #disciplinasECT = df.loc[df['codigo'].str.contains('ECT22') | df['codigo'].str.contains('ECT23') | df['codigo'].str.contains('ECT24'),:]
  return disciplinasECT

##4. Leitura das Bases
<a id='leiturabase'></a>

Urls das notas e da carga horária

In [0]:
notas_url = [
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2015.1.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2015.2.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2016.1.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2016.2.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2017.1.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2017.2.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2018.1.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2018.2.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2019.1.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2019.2.csv'
            ]
carga_horaria = 'http://dados.ufrn.br/dataset/3fea67e8-6916-4ed0-aaa6-9a8ca06a9bdc/resource/9a3521d2-4bc5-4fda-93f0-f701c8a20727/download/componentes-curriculares-presenciais.csv'

Lendo como DataFrame

In [0]:
col = ["periodo","codigo","disciplina","turma","siape","horario","matricula","nota","situacao"]
notas2015_1 = pd.read_csv(notas_url[0], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2015_2 = pd.read_csv(notas_url[1], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2016_1 = pd.read_csv(notas_url[2], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2016_2 = pd.read_csv(notas_url[3], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2017_1 = pd.read_csv(notas_url[4], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2017_2 = pd.read_csv(notas_url[5], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2018_1 = pd.read_csv(notas_url[6], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2018_2 = pd.read_csv(notas_url[7], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2019_1 = pd.read_csv(notas_url[8], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
#notas2019_2 = pd.read_csv(notas_url[9], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)

carga = pd.read_csv(carga_horaria, sep=";")
carga = carga[["codigo","ch_total"]]

#Parte III - Desenvolvimento
<a id='enten_dados'></a>


## 1. Entendimento de Negócio
<a id='negocio'></a>

#### ● Determinar objetivos de Negócio
#### ● Estabelecer Critérios de Sucesso da Iniciativa
#### ● Análise do contexto: terminologia, benefício, riscos e plano de contingência
#### ● Determinar objetivos de Mineração de Dados
#### ● Plano do Projeto


## 2. Entendimento dos Dados Primários
<a id='enten_dados'></a>

### [2.1 Descrição dos Dados Primários](#descricao)
### [2.2 Qualidade dos Dados Primários](#qualidade)
### [2.3 Exploração dos Dados Primários](#exploracao)

###2.1 Descrição dos Dados Primários
<a id='descricao'></a>


#### <center>Dados dos Daframes de Notas:
| Semestre | Número de atributos (colunas) | Número de Registros (linhas) |
| --- | --- | --- |
| 2015.1 | 9 | --- |
| 2015.2 | 9 | |
| 2016.1 | 9 | |
| 2016.2 | 9 | |
| 2017.1 | 9 | |
| 2017.2 | 9 | |
| 2018.1 | 9 | |
| 2018.2 | 9 | |
| 2019.1 | 9 | |


#### <center>Dados do Dataframe de Carga Horária:
| Número de atributos (colunas) | Número de Registros (linhas) |
| --- | --- |
| 2 | 39269 |




####Descrição de Colunas do Dataframe de Notas:

<font color='blue'> <b>1. periodo</b> </font> Período em que a disciplina foi cursada (.1 ou .2)

<font color='blue'> <b>2. codigo</b> </font> Código de todas as disciplinas na UFRN

<font color='blue'> <b>3. disciplina</b> </font> Nome da discplina 

<font color='blue'> <b>4. turma</b> </font> Código da turma

<font color='blue'> <b>5. siape</b> </font> Código que identifica o professor que ministrou a disciplina

<font color='blue'> <b>6. horario</b> </font> Horário e intervalo de datas em que a disciplina foi ministrada

<font color='blue'> <b>7. Matrícula</b> </font> Código identificador do estudante

<font color='blue'> <b>8. Nota</b> </font> Nota do estudante na disciplina

#### Coluna Alvo
<font color='red'> <b>9. Situação</b> </font> Estato final na disciplina

####Descrição de Colunas do Dataframe de Notas:

<font color='blue'> <b>1. Código</b> </font> Código da disciplina

<font color='blue'> <b>2. Ch_total</b> </font> Carga Hóraria da displina

-------------------------------------------------------------------------------

Os Dataframes contêm todos os departamentos da UFRN. Por enquanto o foco do trabalho está em prever o ciclo básico da ECT. Mais na frente será feito uma seleção afim de escolher apenas matérias da ECT.



Exemplos de registros nos 2 tipos de DataFrames

In [0]:
notas2018_1.sample(10)

In [0]:
carga.head(2)

###2.2 Qualidade dos Dados Primários
<a id='qualidade'></a>

Total 489 disciplinas realizadas na UFRN no semestre 15.1

In [0]:
notas2015_1.groupby('codigo')['nome'].count()

Informações sobre valores não nulos, total de registros de todos as bases de dados

Em quase todos os dataframes das notas não contêm valores nulos, com excessão de 16.2, 18.2 e 19.1, onde estão faltando em cada 3 valores correspondentes aos horários das displinas. Ou seja, temos dataframes consistentes

No dataframe de cargas horárias também temos consistência

In [0]:
notas2015_1.info()

In [0]:
notas2015_2.info()

In [0]:
notas2016_1.info()

In [0]:
notas2016_2.info()

In [0]:
notas2017_1.info()

In [0]:
notas2017_2.info()

In [0]:
notas2018_1.info()

In [0]:
notas2018_2.info()

In [0]:
notas2019_1.info()

In [0]:
carga.info()

###2.3 Exploração dos Dados Primários
<a id='exploracao'></a>

Temos várias situações disponíveis ao fim da matéria. Excluída, Trancada, Desistência, Indiferido e Cancelado não farão parte da nossa análise que busca apenas alunos que foram do começo ao fim da matéria.

Todos os tipos de reprovação iram virar uma só, assim como às aprovações.

In [0]:
notas2015_1['situacao'].unique()

Vemos que existe disciplinas em que a nota média final foi '-'. Como pode?

A primeira hipótese seria de que algúem zerou todas as unidades e saiu com nota 0, e o sistema colocaria '-' 

Descobri que existem disciplinas que contam apenas para carga horária complementar. Ou seja, não tem uma nota final para o estudante

Além disso vemos que as notas estão em objeto (string), vai ser necessário a conversão para float

In [0]:
notas2015_1['nota'].unique()

##3. Preparação dos Dados Primários
<a id='preparo'></a>
#### [3.1 Geração de Dados Genéricos](#geracao)
#### [3.2. Concatenação dos Dados Genéricos do semestre anterior com o resultado da disciplina no semestre seguinte](#concatenacao)
#### [3.3. Transformação usando Pipeline](#pipeline)

### 3.1. Geração de Dados Genéricos
<a id='geracao'></a>

Adicionando os DF a uma lista para minimizar o número de linhas

In [0]:
lista_df_notas = [notas2015_1, notas2015_2, notas2016_1, notas2016_2 , notas2017_1, notas2017_2, notas2018_1, notas2018_2, notas2019_1]

In [0]:
lista_df_notas[-1].iloc[:2,1:]

In [25]:
lista_df_notas[-1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20620 entries, 0 to 20619
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   periodo     20620 non-null  int64 
 1   codigo      20620 non-null  object
 2   disciplina  20620 non-null  object
 3   turma       20620 non-null  object
 4   siape       20620 non-null  object
 5   horario     20617 non-null  object
 6   matricula   20620 non-null  int64 
 7   nota        20620 non-null  object
 8   situacao    20620 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.4+ MB


A idéia de geração dos dados genéricos é a seguinte

Passos de transformação de dados

* primeiro
* segundo

Classe para realizar transformação nos Dados

Ela será no formato sugerido pela Sklearn para ser compatível com pipelines


serao 2 classes de transformacao nesse notebook. A que treina os modelos é a proxima

depois o próximo passo é concatenar o dataframe antecedente com o resultado na materia do semestre seguinte 


bagunça.com
Função para concatenar o antecedente com a situação em matéria posterior
def concatenaAntecessoresComResultadoPosterior(tableAntecedente,tableAtual,situacaoPosterior,final,flag):



Vai retornar uma lista de listas, onde vai ser uma com os dados genericos e a outra com as notas contendo todas as transformações antes da geração de dados genéricos

In [28]:
df_nota_generico[1][0].iloc[:,:-3].head(2)

,codigo,disciplina,turma,siape,horario,matricula
0,DEM0304,TERMODINAMICA BASICA,01,"1753416-1,2154326-9",24T56,2014057205
6,ECT2106,"CIÊNCIA, TECNOLOGIA E SOCIEDADE",04,1051570-6,4N12 (02/02/2015 - 20/06/2015),2015000889


In [29]:
df_nota_generico[0][3].head(3)

,matricula,mediaNotas,menorNota,maiorNota,quantidadeDisciplinasCursadas,quantidadeAprovacoes,quantidadeReprovacoes,mediaAprovadas,mediaReprovadas,cargaHoraria,taxaDeSucesso,semestre,anoIngresso,anosMatriculado
0,2009028457,2.733333,0.0,5.6,3,1,2,5.600000,1.3,180,0.333333,2016.2,2009.0,7
1,2009030796,6.225000,0.5,9.1,4,3,1,8.133333,0.5,210,0.750000,2016.2,2009.0,7
2,2009031954,2.940000,1.3,5.9,5,1,4,5.900000,2.2,300,0.200000,2016.2,2009.0,7


### 3.2. Concatenação dos Dados Genéricos do semestre anterior com o resultado da disciplina no semestre seguinte
<a id='concatenacao'></a>



semestre-atual 18.2
semestre-passa 18.1
resultado-atua 18.2

chega na funcao de busca de reprovacao com o ano atual, e so ele, vai com nome de dfCompleto
df_nota_generico com todos os resultados de todos semestres


crio um dataframe temporario

itero na lista os resultados de todos semestres
se o semestre atual for maior que o da iteração, então
verifica se no passado dele tem disciplina com o mesmo codigo que a atual
se tem ele concatena no dfTemporario e repete até acabar a lista de resultados

### 3.3. Transformação usando Pipeline
<a id='pipeline'></a>

Montagem do Pipeline de transformação

In [0]:
pipeline_dados_genericos = Pipeline(steps = [('gera_dados', geraDadosGenericos(carga)),
                                              ('concatena_semestres', concatenaSemestrePassadoComResultadosDoAtual())
                                             ])


Chamada do transformador

In [0]:
pipeline_dados_genericos.transform(lista_df_notas)

Download do dataset

In [0]:
disciplinasECT.to_csv(r"disciplinasECT201520191.csv")

##6. Avaliação dos Modelos
<a id='avaliacao'></a>

### [6.1 Chamada da Função](#chamada)
### [6.2 Teste de Modelo](#teste)

###6.1 Chamada da Função
<a id='chamada'></a>

In [0]:
(df_relatorio, model_param_disciplinas) = avaliar_modelos(df, hyperparameters)

In [0]:
df_relatorio.head()

In [0]:
model_param_disciplinas

###6.2 Teste de Modelo
<a id='teste'></a>

Importando Modelo

In [0]:
modelo_teste = joblib.load('modelo_.pkl')

Aplicando a transformação no teste

In [0]:
num_transf = NumericalTransformer()
df_teste = model.transform(df)

In [0]:
df_teste.head(2)

Realizando teste de um modelo, prevendo probabilidade de ser aprovado

In [0]:
df_teste.iloc[42,:]

In [0]:
df_teste.info()

In [0]:
modelo_teste.predict(df_teste.iloc[42,:])

##7. Conclusão e Recomendações
<a id='conclusao'></a>

##8. Implantação de Modelos
<a id='implantacao'></a>